In [57]:
import pandas as pd
import re
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd

In [58]:
reviews_df = pd.read_csv("HairSalons_reviews_cloud_export_17427.csv")

In [59]:
reviews_df.head()

,Review,Page_Title
0,Love this salon! I was referred over by two of...,Urban Betty - 128 Photos & 537 Reviews - Hair ...
1,Came in for a summer haircut since my hair was...,Urban Betty - 128 Photos & 537 Reviews - Hair ...
2,I recently moved and have been looking for a g...,Urban Betty - 128 Photos & 537 Reviews - Hair ...
3,Y'all stop what you are doing right now and go...,Urban Betty - 128 Photos & 537 Reviews - Hair ...
4,"Love, love, love this place! I saw Holly and s...",Urban Betty - 128 Photos & 537 Reviews - Hair ...


In [60]:
def get_salon_name(row):
    page_title = row["Page_Title"].split(" - ") # split page title by the dash
    return page_title[0] # The salon name is the first of the split

In [61]:
reviews_df["Title"] = reviews_df.apply(get_salon_name, axis=1)

In [62]:
reviews_df.head(5)

,Review,Page_Title,Title
0,Love this salon! I was referred over by two of...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,Urban Betty
1,Came in for a summer haircut since my hair was...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,Urban Betty
2,I recently moved and have been looking for a g...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,Urban Betty
3,Y'all stop what you are doing right now and go...,Urban Betty - 128 Photos & 537 Reviews - Hair ...,Urban Betty
4,"Love, love, love this place! I saw Holly and s...",Urban Betty - 128 Photos & 537 Reviews - Hair ...,Urban Betty


Reshape into a newer, cleaner dataframe to load to the database

In [63]:
reviews_clean = reviews_df.copy()
reviews_clean.drop(["Page_Title"], axis=1, inplace = True)
reviews_clean.head()

,Review,Title
0,Love this salon! I was referred over by two of...,Urban Betty
1,Came in for a summer haircut since my hair was...,Urban Betty
2,I recently moved and have been looking for a g...,Urban Betty
3,Y'all stop what you are doing right now and go...,Urban Betty
4,"Love, love, love this place! I saw Holly and s...",Urban Betty


Once all the data is put into a pandas dataframe, convert that to a SQL database

In [64]:
f=open("db.txt", "r")
contents=0
if not(contents):
    contents = f.read()
username = contents.split("\n")[0]
password = contents.split("\n")[1]
dbname = 'reviews_db'

In [52]:
engine = create_engine('postgres://%s:%s@localhost/%s'%(username,password, dbname))

In [53]:
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [54]:
reviews_clean.to_sql('reviews_data_table', engine, if_exists='replace')

Check to make sure the pandas dataframe made it to the database correctly

In [56]:
con = None
con = psycopg2.connect(database = dbname, user = username, password = password, port=5432, host= "/var/run/postgresql/")

# query:
sql_query = """
SELECT * FROM reviews_data_table;
"""
review_data_from_sql = pd.read_sql_query(sql_query,con)
review_data_from_sql.head()

,index,Review,Title
0,0,Love this salon! I was referred over by two of...,Urban Betty
1,1,Came in for a summer haircut since my hair was...,Urban Betty
2,2,I recently moved and have been looking for a g...,Urban Betty
3,3,Y'all stop what you are doing right now and go...,Urban Betty
4,4,"Love, love, love this place! I saw Holly and s...",Urban Betty
